# Deutsch's Algorithm
---

Deutsch's Algorithm is a simple quantum algorithm that determines whether a given function \( $f: \{0, 1\} \rightarrow \{0, 1\}$ \) is constant or balanced. The algorithm requires only one query to the function, whereas a classical algorithm would require two.

## ⊗ Sequential Code

In [1]:
%%writefile deutsch-sequential.c 
#include <stdio.h>

// Define the function f: {0, 1} -> {0, 1}
int f(int x) {
    // Example function: f(0) = 0, f(1) = 1 (balanced)
    return x;
}

int deutschAlgorithm() {
    // Evaluate f(0) and f(1)
    int f0 = f(0);
    int f1 = f(1);

    // Check if f is constant or balanced
    if (f0 == f1) {
        return 0; // Constant function
    } else {
        return 1; // Balanced function
    }
}

int main(int argc, char **argv) 
{
    int result = deutschAlgorithm();

    if (result == 0) {
        printf("The function is constant.\n");
    } else {
        printf("The function is balanced.\n");
    }

    return 0;
}

Writing deutsch-sequential.c


In [2]:
!gcc deutsch-sequential.c -o deutsch-sequential

In [3]:
!./deutsch-sequential

The function is balanced.


## ⊗ CUDA

In [4]:
%%writefile deutsch-cuda.cu
#include <stdio.h>
#include <cuda.h>

// Define the function f: {0, 1} -> {0, 1}
__device__ int f(int x) {
    // Example function: f(0) = 0, f(1) = 1 (balanced)
    return x;
}

// CUDA kernel to evaluate f(0) and f(1) in parallel
__global__ void deutsch_kernel(int *d_results) {
    int tid = threadIdx.x; // Thread ID (0 or 1)
    d_results[tid] = f(tid); // Evaluate f(x) for x = tid
}

int main(int argc, char **argv) {

    int h_results[2]; // Host results array
    int *d_results;   // Device results array

    // Allocate memory on the device
    cudaMalloc((void **)&d_results, 2 * sizeof(int));

    // Launch the kernel with 2 threads (one for each input)
    deutsch_kernel<<<1, 2>>>(d_results);

    // Copy the results back to the host
    cudaMemcpy(h_results, d_results, 2 * sizeof(int), cudaMemcpyDeviceToHost);

    // Check if the function is constant or balanced
    if (h_results[0] == h_results[1])
        printf("The function is constant.\n");
    else 
        printf("The function is balanced.\n");

    // Free device memory
    cudaFree(d_results);

    return 0;
}

Writing deutsch-cuda.cu


In [5]:
!nvcc deutsch-cuda.cu -o deutsch-cuda

In [6]:
!./deutsch-cuda

The function is balanced.


<details>

  <summary>Tips of the code</summary>

  **`CUDA Execution`**
    
**a) CUDA Kernel (`deutsch_kernel`)**
```cpp
__global__ void deutsch_kernel(int *d_results) {
    int tid = threadIdx.x;   // Thread ID (0 or 1)
    d_results[tid] = f(tid); // Evaluate f(x) for x = tid
}
```
- This kernel is launched with **two threads** (one per input: `tid = 0` and `tid = 1`).
- Each thread computes \( f(x) \) for its corresponding input and stores the result in `d_results`.

**b) Device Function (`f`)**
```cpp
__device__ int f(int x) {
    return x; // Example: f(0) = 0, f(1) = 1 (balanced function)
}
```
- This defines the function \( f(x) \).
- In this example, \( f(0) = 0 \) and \( f(1) = 1 \), which makes the function **balanced**.


 **`Host Code Execution`**
    
 **a) Memory Allocation**
```cpp
cudaMalloc((void **)&d_results, 2 * sizeof(int));
```
- Allocates memory on the **GPU** to store results for \( f(0) \) and \( f(1) \).

 **b) Kernel Launch**
```cpp
deutsch_kernel<<<1, 2>>>(d_results);
```
- **1 block** with **2 threads** is launched.
- Thread 0 computes \( f(0) \) and stores it in `d_results[0]`.
- Thread 1 computes \( f(1) \) and stores it in `d_results[1]`.

 **c) Copying Data Back to Host**
```cpp
cudaMemcpy(h_results, d_results, 2 * sizeof(int), cudaMemcpyDeviceToHost);
```
- Copies the computed results from the **GPU** back to the **CPU**.

 **d) Checking Function Type**
```cpp
if (h_results[0] == h_results[1]) 
    printf("The function is constant.\n");
else
    printf("The function is balanced.\n");
```
- If \( f(0) = f(1) \), the function is **constant**.
- Otherwise, it is **balanced**.

</details>

## ⊗ CUDA-Q `Python`

In [7]:
%%writefile deutsch-cudaq.py
import cudaq
import numpy as np
from typing import List

cudaq.set_target("nvidia")

# Here we input the values of [f(0), f(1)] which allows us to represent constant or balanced functions.
fx = [0, 1]

# Let us now code up the circuit shown above following the state Psi after each step.
qubit_count = 2

@cudaq.kernel
def kernel(fx: List[int]):
    qubit_0 = cudaq.qubit()
    qubit_1 = cudaq.qubit()

    # Psi 0
    x(qubit_1)

    # Psi 1
    h(qubit_0)
    h(qubit_1)

    # Psi 2 - oracle
    if fx[0] == 1:
        x.ctrl(qubit_0, qubit_1)
        x(qubit_1)

    if fx[1] == 1:
        x.ctrl(qubit_0, qubit_1)

    # Psi 3
    h(qubit_0)

    # Measure the qubit to yield if the function is constant or balanced.
    mz(qubit_0)

print(cudaq.draw(kernel, fx))

result = cudaq.sample(kernel, fx, shots_count=1)

if np.array(result)[0] == '0':
    print('The function is constant.')
elif np.array(result)[0] == '1':
    print('The function is balanced.')

Writing deutsch-cudaq.py


In [8]:
!python deutsch-cudaq.py

     ╭───╮          ╭───╮
q0 : ┤ h ├───────●──┤ h ├
     ├───┤╭───╮╭─┴─╮╰───╯
q1 : ┤ x ├┤ h ├┤ x ├─────
     ╰───╯╰───╯╰───╯     

The function is balanced.


<details>

  <summary>Tips of the code</summary>

**`Step-by-Step Quantum Computation`**
    
 **(a) Register Initialization**
```python
qubit_0 = cudaq.qubit()
qubit_1 = cudaq.qubit()
```
- **qubit_0**: Represents the **input qubit** (superposition state).
- **qubit_1**: Represents the **oracle qubit**, initialized in \(|1\rangle\).

**(b) Prepare Oracle Qubit**
```python
x(qubit_1)  # Apply X gate on qubit_1 (flips |0> to |1>)
```
- This initializes qubit_1, which is needed for phase kickback.

**(c) Apply Hadamard Gates to Create Superposition**
```python
h(qubit_0)
h(qubit_1)
```
- Hadamard (\( H \)) is applied to **both qubits**

**(d) Oracle Implementation**
```python
if fx[0] == 1:
    x.ctrl(qubit_0, qubit_1)
    x(qubit_1)

if fx[1] == 1:
    x.ctrl(qubit_0, qubit_1)
```
- The oracle **encodes** \( f(x) \) in qubit_1 using **controlled-X gates**.
- Depending on \( f(0) \) and \( f(1) \), the circuit applies:
  - **\( X \) (NOT gate)** on qubit_1 if \( f(0) = 1 \).
  - **Controlled-X (CNOT) gate** if \( f(1) = 1 \).
- This step **modifies qubit_0's phase based on the function**.

**(e) Interference via Hadamard**
```python
h(qubit_0)
```
- Applying \( H \) again on qubit_0 **causes interference**, leading to:
  - If \( f(0) = f(1) \) → **qubit_0 collapses to |0⟩** (constant function).
  - If \( f(0) \neq f(1) \) → **qubit_0 collapses to |1⟩** (balanced function).

**(f) Measurement**
```python
mz(qubit_0)
```
- Measures qubit_0 to determine if the function is **constant or balanced**.


**`Running the Quantum Kernel`**
    
**(a) Visualizing the Quantum Circuit**
```python
print(cudaq.draw(kernel, fx))
```
This prints the circuit representation.

**(b) Executing and Checking Result**
```python
result = cudaq.sample(kernel, fx, shots_count=1)
```
- Runs the quantum kernel **once** and samples the result.

**(c) Interpreting the Output**
```python
if np.array(result)[0] == '0':
    print('The function is constant.')
elif np.array(result)[0] == '1':
    print('The function is balanced.')
```
- If qubit_0 measures **0**, \( f(x) \) is **constant**.
- If qubit_0 measures **1**, \( f(x) \) is **balanced**.

</details>

## ⊗ CUDA-Q `C++`

In [1]:
%%writefile deutsch-cudaq.cpp
#include <cudaq.h>
#include <iostream>
#include <vector>

__qpu__ void kernel(int N) {
  cudaq::qvector q(N);

  x(q[0]);  // Apply X gate to the ancilla qubit

  // Apply Hadamard gates to both qubits
  h(q[0]);
  h(q[1]);

  x<cudaq::ctrl>(q[0], q[1]);
      
  h(q[0]);

  // Measure the first qubit
  mz(q[0]);
}

int main(int argc, char **argv) {
    int qubit_count = 2;
    auto result_0 = cudaq::sample(kernel, qubit_count);
    result_0.dump();

    auto result = std::stoi(result_0.most_probable());

    // Output the result
    if (result == 0) {
    std::cout << "The function is constant." << std::endl;
    } else {
    std::cout << "The function is balanced." << std::endl;
    }

    return 0;
}

Writing deutsch-cudaq.cpp


In [2]:
!nvq++ deutsch-cudaq.cpp -o deutsch-cudaq

In [3]:
!./deutsch-cudaq

{ 1:1000 }
The function is balanced.


<details>

<summary>Tips of the code</summary>

**`Step-by-Step Breakdown of the CUDA-Q Kernel`**
    
```cpp
__qpu__ void kernel(int N) {
  cudaq::qvector q(N);
```
- **Declares a quantum vector \( q \)** of size **N** (which is 2 in this case).
- Each entry represents a **quantum bit (qubit)**.

**(a) Step 1: Initialize the Oracle Qubit**
```cpp
  x(q[0]);  // Apply X gate to the ancilla qubit
```
- Applies an **\( X \) (NOT) gate** on **q[0]**.
- This flips the qubit.
- This is necessary for the **phase kickback** effect in the algorithm.

**(b) Step 2: Apply Hadamard Gates (Superposition)**
```cpp
  h(q[0]);
  h(q[1]);
```
- The **Hadamard gate \( H \)** puts each qubit into an **equal superposition.**
- Now, the system is in a **superposition of all possible inputs**.

**(c) Step 3: Apply the Oracle Function**
```cpp
  x<cudaq::ctrl>(q[0], q[1]);
```
- **Controlled-X (CNOT) Gate**: Applies an \( X \) gate on **q[1]** **only if** **q[0]** is **1**.
- This implements the **oracle \( U_f \)**, encoding the function's behavior into the quantum state.


**(d) Step 4: Apply Hadamard Again (Interference)**
```cpp
  h(q[0]);
```
- Another **Hadamard transform** is applied to **q[0]**.
- This **interferes** the quantum states, extracting whether \( f(x) \) is **constant or balanced**:
  - If \( f(x) \) is **constant**, the interference collapses **q[0]** to \(|0\rangle\).
  - If \( f(x) \) is **balanced**, **q[0]** collapses to \(|1\rangle\).

**(e) Step 5: Measure the First Qubit**
```cpp
  mz(q[0]);
```
- Measures **q[0]** to get **0 or 1**.
- If the result is **0**, \( f(x) \) is **constant**.
- If the result is **1**, \( f(x) \) is **balanced**.


**`Running the Kernel in the Main Function`**
```cpp
int main(int argc, char **argv) {
    int qubit_count = 2;
    auto result_0 = cudaq::sample(kernel, qubit_count);
```
- Calls the **quantum kernel** and samples the result.

```cpp
    result_0.dump();
```
- Prints the measurement results.

```cpp
    auto result = std::stoi(result_0.most_probable());
```
- Extracts the most probable measurement outcome (0 or 1).

```cpp
    if (result == 0)
        std::cout << "The function is constant." << std::endl;
    else 
        std::cout << "The function is balanced." << std::endl;
```
- **Interprets the result**:
  - If the measured qubit **q[0] = 0**, the function is **constant**.
  - If the measured qubit **q[0] = 1**, the function is **balanced**.
</details>

## Clear the Memory

Before moving on, please execute the following cell to clear up the CPU memory. This is required to move on to the next notebook.

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)